## Introduction

In this assignment, you are going to build a classifier for named entities from the Groningen Meaning Bank corpus.  Named entity recognition (NER) takes noun phrases from a text and identifies whether they are persons, organizations, and so on.  You will be using the Groningen Meaning Bank named entity corpus available on mltgpu at `/scratch/lt2222-v21-resources/GMB_dataset.txt`.  In this version of the task, you will assume we know *that* something is a named entity, and instead use multi-class classification to identify its type.  So you will be doing named entity classification but *not* recognition.

The data looks like this: 

```
3996    182.0   Nicole  NNP     B-per
3997    182.0   Ritchie NNP     I-per
3998    182.0   is      VBZ     O
3999    182.0   pregnant        JJ      O
4000    182.0   .       .       O
4001    183.0   Speaking        VBG     O
4002    183.0   to      TO      O
4003    183.0   ABC     NNP     B-org
4004    183.0   News    NNP     I-org
4005    183.0   interviewer     NN      O
4006    183.0   Dianne  NNP     B-per
4007    183.0   Sawyer  NNP     I-per
4008    183.0   ,       ,       O
4009    183.0   the     DT      O
4010    183.0   25-year-old     JJ      O
4011    183.0   co-star NN      O
4012    183.0   of      IN      O
4013    183.0   TV      NN      O
4014    183.0   's      POS     O
4015    183.0   The     DT      B-art
4016    183.0   Simple  NNP     I-art
4017    183.0   Life    NNP     I-art
4018    183.0   said    VBD     O
4019    183.0   she     PRP     O
4020    183.0   is      VBZ     O
4021    183.0   almost  RB      O
4022    183.0   four    CD      O
4023    183.0   months  NNS     O
4024    183.0   along   IN      O
4025    183.0   in      IN      O
4026    183.0   her     PRP$    O
4027    183.0   pregnancy       NN      O
4028    183.0   .       .       O
```

The first column is the line number.  The second column is a sentence number (for some reason given as a float; ignore it).  The third column is the word.  The fourth column is a part of speech (POS) tag in Penn Treebank format.  The last column contains the named entity annotation. 

The annotation works like this.  Every `O` just means that the row does not represent a named entity.  `B-xyx` means the first word in a named entity with type `xyx`. `I-xyz` means the second and later words of an `xyz` entity, if there are any.  That means that every time there's a `B` or an `I`, there's a named entity.  

The entity types in the corpus are `art`,
`eve`,
`geo`,
`gpe`,
`nat`,
`org`,
`per`,
and `tim`

Your task is the following.

1. To preprocess the text (lowercase and lemmatize; punctuation can be preserved as it gets its own rows).

2. To create instances from every from every identified named entity in the text with the type of the NE as the class, and a surrounding context of five words on either side as the features.  

3. To generate vectors and split the instances into training and testing datasets at random.

4. To train a support vector machine (via `sklearn.svm.LinearSVC`) for classifying the NERs.

5. To evaluate the performance of the classifier.


You will do this by modifying a separate file containing functions that will be called from this notebook as a module.  You can modify this notebook for testing purposes but please only submit the original.  You will document everything in Markdown in README.md and submit a GitHub repository URL.

This assignment is due on **Tuesday, 2021 March 9 at 23:59**.  It has **25 points** and **7 bonus points**.

In [1]:
import a2
from sklearn.svm import LinearSVC



[nltk_data] Downloading package wordnet to
[nltk_data]     /home/gussarayo@GU.GU.SE/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
gmbfile = open('/scratch/lt2222-v21-resources/GMB_dataset.txt', "r")

## Part 1 - preprocessing (3 points)

See step 1 above.  The data is coming to you as an unused file handle object.  You can return the data in any indexable form you like.  You can also choose to remove infrequent or uninformative words to reduce the size of the feature space. (Document this in README.md.)

In [3]:

inputdata = a2.preprocess(gmbfile)
gmbfile.close()
inputdata[20:40]

,Line_No,Sentence_No,Word,POS,NER
20,20,1,from,IN,O
21,21,1,that,DT,O
22,22,1,country,NN,O
23,24,2,family,NNS,O
24,25,2,of,IN,O
25,26,2,soldier,NNS,O
26,27,2,kill,VBN,O
27,28,2,in,IN,O
28,29,2,the,DT,O
29,30,2,conflict,NN,O


## Part 2 - Creating instances (7 points)

Do step 2 above.  You will create a collection of Instance objects.  Remember to consider the case where the NE is at the beginning of a sentence or at the end, or close to either (you can create a special start token for that).  You can also start counting from before the `B` end of the NE mention and after the last `I` of the NE mention. That means that the instances should include things before and after the named entity mention, but not the named entity text itself.

In [4]:

instances = a2.create_instances(inputdata)

In [5]:
instances[20:40]

[Class: gpe Features: ['<S5>', 'this', 'week', 'restart', 'part', 'the', 'conversion', 'process', 'at', '</S5>'],
 Class: geo Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S5>', 'official', 'say', 'they', 'expect', 'get', 'access', 'to', 'sealed', 'sensitive'],
 Class: tim Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'surveillance', 'begin', 'function', '</S5>', '</S4>', '</S3>'],
 Class: org Features: ['<S5>', 'surveillance', 'system', 'begin', 'function', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: org Features: ['<S2>', '<S3>', '<S4>', '<S5>', 'backing', 'threaten', 'to', 'refer', '</S5>', '</S4>'],
 Class: gpe Features: ['<S5>', 'backing', 'have', 'threaten', 'to', '</S5>', '</S4>', '</S3>', '</S2>', '</S1>'],
 Class: gpe Features: ['<S1>', '<S2>', '<S3>', '<S4>', '<S5>', 'which', 'could', 'impose', 'sanction', 'if'],
 Class: org Features: ['<S3>', '<S4>', '<S5>', 'which', 'could', 'sanction', 'if',

## Part 3 - Creating the table and splitting (10 points)

Here you're going to write the functions that create a data table with "document" vectors representing each instance and split the table into training and testing sets and random with an 80%/20% train/test split.

In [6]:

bigdf = a2.create_table(instances)
bigdf[20:40]

,Class,0,1,2,3,4,5,6,7,8,...,290,291,292,293,294,295,296,297,298,299
20,gpe,5.032928,-1.120715,0.375585,0.116922,0.145641,-0.487438,-0.001267,-0.004255,0.001975,...,-0.000006,9.855534e-07,-1.341620e-06,0.000004,0.000011,-0.000015,-0.000021,-0.000005,-1.300866e-05,-0.000002
21,gpe,5.989315,0.056117,-0.157471,-0.094129,-0.125747,-0.019278,-0.004521,-0.022386,0.034266,...,-0.001249,-1.538678e-03,1.329189e-03,0.000881,0.000806,0.000655,-0.003194,0.001694,6.858206e-05,-0.000749
22,gpe,4.991326,0.047065,-0.132388,-0.079418,-0.106476,-0.016392,-0.003859,-0.019125,0.029345,...,-0.000276,-3.582937e-04,3.072175e-04,0.000203,0.000207,0.000150,-0.000743,0.000400,2.448824e-05,-0.000176
23,gpe,5.030752,0.568323,0.758935,0.168941,0.158547,0.022137,-0.021922,0.016067,-0.048622,...,-0.000396,-1.200539e-04,1.742763e-04,0.000205,-0.000208,-0.000198,-0.000321,0.000163,-1.514129e-04,0.000739
24,gpe,7.997703,0.133758,-0.194957,-0.166327,-0.280392,-0.060328,0.018610,-0.101562,0.285300,...,-0.002300,-2.645838e-03,6.293904e-04,-0.004339,-0.003516,0.004062,0.008540,-0.009203,5.750378e-03,-0.001111
25,gpe,6.028335,-0.904722,0.224634,0.021182,-0.067715,0.884588,0.006081,0.025655,-0.028096,...,-0.000011,-8.508483e-07,5.114984e-09,0.000008,0.000018,-0.000021,-0.000036,-0.000005,-1.908247e-05,-0.000004
26,gpe,6.000123,0.074781,-0.239641,-0.181075,-0.329846,-0.087698,-0.062631,-0.550162,-0.670491,...,-0.000011,-1.518608e-05,1.311504e-05,0.000009,0.000009,0.000006,-0.000031,0.000017,9.115533e-07,-0.000008
27,gpe,5.998953,0.072186,-0.226844,-0.164635,-0.280608,-0.063379,-0.026210,-0.155031,0.548019,...,-0.000013,-1.700625e-05,1.468650e-05,0.000010,0.000010,0.000007,-0.000035,0.000019,1.021444e-06,-0.000008
28,gpe,6.012878,0.124895,-0.685803,0.906508,0.261858,0.018889,0.002698,0.011721,-0.011259,...,0.000010,6.345566e-05,-1.186579e-05,-0.000011,0.000008,-0.000027,-0.000106,-0.000025,7.237054e-05,-0.000110
29,gpe,6.008186,0.099747,-0.404621,-0.711291,0.869033,0.038004,0.005342,0.023739,-0.027885,...,-0.000017,-5.538719e-05,2.865854e-05,-0.000055,-0.000061,0.000097,0.000013,0.000084,-2.720647e-05,0.000018


In [7]:
train_X, train_y, test_X, test_y = a2.ttsplit(bigdf)

# X and y mean feature matrix and class respectively.
train_X, train_y, test_X, test_y

(array([[ 2.39700000e+04,  5.00861132e+00,  8.79692931e-02, ...,
          8.35827654e-05, -2.88312724e-05,  2.15657469e-05],
        [ 6.68250000e+04,  6.99079521e+00,  8.55250670e-02, ...,
          2.07168327e-03, -1.08238816e-04,  7.44886344e-04],
        [ 5.91520000e+04,  4.99615416e+00,  6.17300500e-02, ...,
          8.11853014e-03, -1.24681696e-03,  4.56520278e-03],
        ...,
        [ 1.73580000e+04,  6.02833453e+00, -9.04722127e-01, ...,
         -5.39032500e-06, -1.90824711e-05, -3.78032220e-06],
        [ 5.14580000e+04,  6.02833453e+00, -9.04722127e-01, ...,
         -5.39032500e-06, -1.90824711e-05, -3.78032220e-06],
        [ 5.98600000e+04,  5.00861132e+00,  8.79692931e-02, ...,
          8.35827654e-05, -2.88312724e-05,  2.15657469e-05]]),
 0        gpe
 1        geo
 2        geo
 3        art
 4        geo
         ... 
 55343    tim
 55344    tim
 55345    per
 55346    art
 55347    tim
 Name: Class, Length: 55348, dtype: object,
 array([[ 1.20000000e+01,  7.02

In [8]:
len(test_y) / (len(test_y) + len(train_y))

0.2

In [9]:
len(test_X) / (len(test_X) + len(train_X))

0.2

In [10]:
test_y[0]

'geo'

## Part 4 - Training the model (0 points)

This part you won't do yourself.

In [11]:
model = LinearSVC()
model.fit(train_X, train_y)
train_predictions = model.predict(train_X)
test_predictions = model.predict(test_X)

/usr/local/lib64/python3.7/site-packages/sklearn/svm/_base.py:986: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
train_predictions

array(['geo', 'per', 'per', ..., 'geo', 'per', 'per'], dtype=object)

In [13]:
train_y

0        gpe
1        geo
2        geo
3        art
4        geo
        ... 
55343    tim
55344    tim
55345    per
55346    art
55347    tim
Name: Class, Length: 55348, dtype: object

In [14]:
test_predictions

array(['geo', 'geo', 'geo', ..., 'per', 'per', 'per'], dtype=object)

In [15]:
test_y

0        geo
1        geo
2        gpe
3        gpe
4        per
        ... 
13832    tim
13833    tim
13834    tim
13835    tim
13836    per
Name: Class, Length: 13837, dtype: object

## Part 5 - Evaluation (5 points)

Investigate for yourself what a "confusion matrix".  Then implement a function that takes the data and produces a confusion matrix in any readable form that allows us to compare the performance of the model by class.  

In [16]:
a2.confusion_matrix(test_y, test_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,0,0,69,1,0,6,20,0
eve,2,2,34,0,0,8,45,0
geo,128,96,1541,7,0,522,1834,0
gpe,61,58,1213,18,0,253,948,0
nat,0,0,23,0,0,3,10,0
org,58,58,983,8,0,339,963,0
per,79,82,779,5,0,327,972,0
tim,67,58,896,9,0,252,1000,0


In [17]:
a2.confusion_matrix(train_y, train_predictions)

,art,eve,geo,gpe,nat,org,per,tim
art,5,6,324,0,0,35,64,0
eve,6,4,170,0,0,29,150,0
geo,537,424,6154,32,0,1901,7524,0
gpe,210,232,4559,68,0,1028,3652,0
nat,3,3,103,1,0,15,39,0
org,250,256,4126,50,0,1332,3937,0
per,283,300,3161,20,0,1271,3791,0
tim,262,267,3662,26,0,1084,3992,0


Examine the matrix and describe your observations in README.md.  In particular, what do you notice about the predictions on the training data compared to those on the test data.

## Bonus Part A - Error analysis (2 points)

Look at the weakest-performing classes in the confusion matrix (or any, if they all perform poorly to the same extent).  Find some examples in the test data on which the classifier classified incorrectly for those classes.  What do you think is the reason why those are hard?  Consider linguistic factors and statistical factors, if applicable.  Write your answer in README.md.

## Bonus Part B - Expanding the feature space (7 points)

Run the entire process above, but incorporate part-of-speech tag information into the feature vectors.  It's your choice as to how to do this, but document it in README.md.  Your new process should run from the single call below:

In [18]:
a2.bonusb('/scratch/lt2222-v21-resources/GMB_dataset.txt')